In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
c1=pd.read_excel("HW2.xlsx",sheet_name="production cost")
periods = list(c1.iloc[:,0])
c1=dict(zip(c1.time, c1.cost))
c2=pd.read_excel("HW2.xlsx",sheet_name="procurement cost")
Y = dict(zip(c2.time, c2.Y_t))
c2=dict(zip(c2.time, c2.cost))
c4=pd.read_excel("HW2.xlsx",sheet_name="fixed operatin cost")
c4=dict(zip(c4.time, c4.cost))
d=pd.read_excel("HW2.xlsx",sheet_name="demand")
d=dict(zip(d.time, d.demand))
params=pd.read_excel("HW2.xlsx",sheet_name="other parameters",header=None)
K = params.iloc[0,1]
c3 = params.iloc[1,1]
h = params.iloc[2,1]

### Question 3, part C

In [68]:
#model
m = gp.Model("q3")
m.ModelSense = GRB.MINIMIZE
#decision variables 
x=m.addVars(periods, name="x", vtype=GRB.CONTINUOUS)
y=m.addVars(periods, name="y",vtype=GRB.CONTINUOUS)
I=m.addVars(periods+[53], name="I", vtype=GRB.CONTINUOUS, lb = 0.0)
z=m.addVars(periods, name="z", vtype=GRB.BINARY)
j=m.addVar(vtype=GRB.BINARY) ## whether we do in-house production or not

In [69]:
m.addConstrs((I[t+1] == I[t] + x[t] + y[t] - d[t]) for t in periods) ## inventory level constraint

m.addConstrs((x[t] <= z[t] * K) for t in periods) ## production capacity

m.addConstrs((y[t] <= (1-z[t]) * Y[t]) for t in periods) ## procurement capacity for period t

m.addConstrs((j >= z[t]) for t in periods) ## inhouse production binary j 

m.addConstrs((z[t] >= z[t-1]) for t in periods[1:])  ## inhouse production is always 1 after its opened at period t

m.addConstr(I[1] == 0) ## initial inventory is zero
 
#m.addConstr(I[53] == 0) ## its not given but i assume that ending inventory should also be zero 
## if i do not add this production at t=52 comes up zero

m.setObjective(gp.quicksum(c1[t] * x[t] + c2[t] * y[t] + h * I[t] + c4[t] * z[t] for t in periods) + c3 * j) ## objective

In [70]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 260 rows, 210 columns and 623 nonzeros
Model fingerprint: 0x5320b315
Variable types: 157 continuous, 53 integer (53 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e-01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 4e+02]
Found heuristic solution: objective 57624.900000
Presolve removed 53 rows and 3 columns
Presolve time: 0.01s
Presolved: 207 rows, 207 columns, 516 nonzeros
Variable types: 155 continuous, 52 integer (52 binary)

Root relaxation: objective 5.242639e+04, 235 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 52426.3904    0   52 57624.9000 52426.3904  9.02%     -    0s
H    0     0                   

In [71]:
for i in range(1,53):
    if x[i].X != 0:
        print(x[i])

<gurobi.Var x[12] (value 86.0)>
<gurobi.Var x[13] (value 102.0)>
<gurobi.Var x[14] (value 121.0)>
<gurobi.Var x[15] (value 143.0)>
<gurobi.Var x[16] (value 168.0)>
<gurobi.Var x[17] (value 157.0)>
<gurobi.Var x[18] (value 151.0)>
<gurobi.Var x[19] (value 150.0)>
<gurobi.Var x[20] (value 141.0)>
<gurobi.Var x[21] (value 126.0)>
<gurobi.Var x[22] (value 100.0)>
<gurobi.Var x[23] (value 80.0)>
<gurobi.Var x[24] (value 96.0)>
<gurobi.Var x[25] (value 96.0)>
<gurobi.Var x[26] (value 86.0)>
<gurobi.Var x[27] (value 86.0)>
<gurobi.Var x[28] (value 108.0)>
<gurobi.Var x[29] (value 86.0)>
<gurobi.Var x[30] (value 86.0)>
<gurobi.Var x[31] (value 66.0)>
<gurobi.Var x[32] (value 67.0)>
<gurobi.Var x[33] (value 69.0)>
<gurobi.Var x[34] (value 42.0)>
<gurobi.Var x[35] (value 49.0)>
<gurobi.Var x[36] (value 53.0)>
<gurobi.Var x[37] (value 46.0)>
<gurobi.Var x[38] (value 26.0)>
<gurobi.Var x[39] (value 3.0)>
<gurobi.Var x[40] (value 7.0)>
<gurobi.Var x[41] (value 81.0)>
<gurobi.Var x[43] (value 67.0)>

In [72]:
for i in range(1,53):
    if y[i].X != 0:
        print(y[i])

<gurobi.Var y[1] (value 49.999999999999886)>
<gurobi.Var y[2] (value 67.0)>
<gurobi.Var y[3] (value 96.0)>
<gurobi.Var y[4] (value 89.0)>
<gurobi.Var y[5] (value 66.0)>
<gurobi.Var y[6] (value 56.0)>
<gurobi.Var y[7] (value 66.0)>
<gurobi.Var y[8] (value 56.0)>
<gurobi.Var y[9] (value 68.0)>
<gurobi.Var y[10] (value 84.0)>
<gurobi.Var y[11] (value 56.0)>


In [73]:
for i in range(1,53):
    if I[i].X != 0:
        print(I[i])

<gurobi.Var I[42] (value 51.0)>
<gurobi.Var I[45] (value 105.0)>


In [74]:
for i in range(1,53):
    if z[i].X != 0:
        print(z[i])

<gurobi.Var z[12] (value 1.0)>
<gurobi.Var z[13] (value 1.0)>
<gurobi.Var z[14] (value 1.0)>
<gurobi.Var z[15] (value 1.0)>
<gurobi.Var z[16] (value 1.0)>
<gurobi.Var z[17] (value 1.0)>
<gurobi.Var z[18] (value 1.0)>
<gurobi.Var z[19] (value 1.0)>
<gurobi.Var z[20] (value 1.0)>
<gurobi.Var z[21] (value 1.0)>
<gurobi.Var z[22] (value 1.0)>
<gurobi.Var z[23] (value 1.0)>
<gurobi.Var z[24] (value 1.0)>
<gurobi.Var z[25] (value 1.0)>
<gurobi.Var z[26] (value 1.0)>
<gurobi.Var z[27] (value 1.0)>
<gurobi.Var z[28] (value 1.0)>
<gurobi.Var z[29] (value 1.0)>
<gurobi.Var z[30] (value 1.0)>
<gurobi.Var z[31] (value 1.0)>
<gurobi.Var z[32] (value 1.0)>
<gurobi.Var z[33] (value 1.0)>
<gurobi.Var z[34] (value 1.0)>
<gurobi.Var z[35] (value 1.0)>
<gurobi.Var z[36] (value 1.0)>
<gurobi.Var z[37] (value 1.0)>
<gurobi.Var z[38] (value 1.0)>
<gurobi.Var z[39] (value 1.0)>
<gurobi.Var z[40] (value 1.0)>
<gurobi.Var z[41] (value 1.0)>
<gurobi.Var z[42] (value 1.0)>
<gurobi.Var z[43] (value 1.0)>
<gurobi.